In [74]:
import pandas as pd
import json

## FTA 발언을 담고 있는 csv 파일과 18대 19대 국회의원에 대한 정보를 담고 있는 자료를 불러온다

data = pd.read_csv("CongressLogFTA.csv")
con18 = pd.read_excel("18con.xlsx")
con19 = pd.read_excel("19con.xlsx")
china = pd.read_excel("chinaname_0524.xlsx")


In [2]:
## 18대 19대 국회의원의 발언만으로 구분하고
## 18대 19대 국회의원의 명단을 준비한다.

data_18 = data[data["ROUND1"] == '18대']
data_19 = data[data["ROUND1"] == '19대']
congressmen_18 = con18["name"].unique()
congressmen_19 = con19["name"].unique()

In [4]:
def filterCongressman(speaker_name, congressmen_list):
    there_is = []
    for idx, sn in enumerate(speaker_name):
        for cn in congressmen_list:  
            if sn.find(cn) != -1 :
                there_is.append((idx, sn, cn))
    return there_is

filtered_18 = filterCongressman(data_18['NAME'], congressmen_18)
filtered_19 = filterCongressman(data_19['NAME'], congressmen_19)

print(f"FTA가 발언된 총 발언 횟수는 {len(data)}개 그 중 18대 국회의 발언은 {len(data_18)} 개 이며 19대 국회의 발언은 {len(data_19)} 개이다.")
print(f"18대 국회의원의 수는 {len(congressmen_18)}이며 19대 국회의원의 수는 {len(congressmen_19)}이다. 이 국회의원 명단을 바탕으로 국회의원의 발언만을 필터링 한다.")
print(f"그 결과 {len(data_18)}개의 18대 국회의 발언 중 18대 국회의원의 발언은 {len(filtered_18)}개 였으며, {len(data_19)}개의 19대 국회의 발언 중 19대 국회의원의 발언은 {len(filtered_19)}개 였다.")

FTA가 발언된 총 발언 횟수는 8444개 그 중 18대 국회의 발언은 5072 개 이며 19대 국회의 발언은 3372 개이다.
18대 국회의원의 수는 330이며 19대 국회의원의 수는 336이다. 이 국회의원 명단을 바탕으로 국회의원의 발언만을 필터링 한다.
그 결과 5072개의 18대 국회의 발언 중 18대 국회의원의 발언은 3285개 였으며, 3372개의 19대 국회의 발언 중 19대 국회의원의 발언은 2121개 였다.


In [27]:
filtering_18 = pd.DataFrame(filtered_18).iloc[:,:3]
filtering_19 = pd.DataFrame(filtered_19).iloc[:,:3]
data_18_filtered = data_18.iloc[list(filtering_18[0]),:]
data_18_filtered["congressman"] = list(filtering_18[2])
data_19_filtered = data_19.iloc[list(filtering_19[0]),:]
data_19_filtered["congressman"] = list(filtering_19[2])
data_filtered = pd.concat([data_18_filtered, data_19_filtered])

<ipython-input-27-27689e8ba06d>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_18_filtered["congressman"] = list(filtering_18[2])
<ipython-input-27-27689e8ba06d>:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data_19_filtered["congressman"] = list(filtering_19[2])


In [48]:
numberOfSpeaks_20 = data_filtered["congressman"].value_counts()[:20]

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20.index, y=numberOfSpeaks_20.values, title="18대 19대 국회에서 FTA 관련 발언을 많이 한 상위 20명의 발언 수")
fig.show()

In [38]:
def makeKoreanName(nameList, congListChina, congListKorea):
    cleanName = []

    for name in nameList:
        for idxCong, cong in enumerate(congListChina):
            if name.find(cong) != -1:
                cleanName.append(congListKorea[idxCong])
                break
            elif idxCong == len(congListChina) - 1:
                cleanName.append(name)
                break

    return cleanName
   
data_18["KoreanName"] = makeKoreanName(list(data_18['NAME']), china["chinese"], china["korean"])
data_19["KoreanName"] = makeKoreanName(list(data_19['NAME']), china["chinese"], china["korean"])

<ipython-input-38-f2c5cc795e27>:15: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-38-f2c5cc795e27>:16: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy



In [39]:
def makeCleanName(nameList, congList):
    cleanName = []

    for name in nameList:
        for idxCong, cong in enumerate(congList):
            if name.find(cong) != -1:
                if cong == "윤영":
                    cleanName.append(name)
                    break
                cleanName.append(cong)
                break
            if idxCong == len(congList) - 1:
                cleanName.append(name)
                break

    return cleanName

def makeCongOnly(nameList, congList):
    cleanName = []

    for name in nameList:
        for idxCong, cong in enumerate(congList):
            if name.find(cong) != -1:
                if cong == "윤영":
                    cleanName.append(None)
                    break
                cleanName.append(cong)
                break
            if idxCong == len(congList) - 1:
                cleanName.append(None)
                break

    return cleanName

data_18["CleanName"] = makeCleanName(list(data_18['KoreanName']), congressmen_18)
data_19["CleanName"] = makeCleanName(list(data_19['KoreanName']), congressmen_19)
data_18["CongOnly"] = makeCongOnly(list(data_18['KoreanName']), congressmen_18)
data_19["CongOnly"] = makeCongOnly(list(data_19['KoreanName']), congressmen_19)

<ipython-input-39-f791d8261a6a>:35: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-39-f791d8261a6a>:36: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-39-f791d8261a6a>:37: SettingWithCopyWarning:


A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy

<ipython-input-39-f791

In [40]:
data_18[["NAME", "KoreanName", "CleanName", "CongOnly"]]



,NAME,KoreanName,CleanName,CongOnly
2910,委員長 秋美愛,추미애,추미애,추미애
2911,진술인 이영희,진술인 이영희,진술인 이영희,None
2912,위원장 박진,위원장 박진,박진,박진
2913,김영록 위원,김영록 위원,김영록,김영록
2914,최규성 위원,최규성 위원,최규성,최규성
...,...,...,...,...
8439,기획재정부장관 윤증현,기획재정부장관 윤증현,기획재정부장관 윤증현,None
8440,관세청장 윤영선,관세청장 윤영선,관세청장 윤영선,None
8441,관세청장 윤영선,관세청장 윤영선,관세청장 윤영선,None
8442,기획재정부장관 윤증현,기획재정부장관 윤증현,기획재정부장관 윤증현,None


In [41]:
data0524 = pd.concat([data_18, data_19])
data0524.to_csv("FTA_congressmen_0524.csv")

In [111]:
data0524 = pd.read_csv("FTA_congressmen_0524.csv")

In [112]:
data_18 = data0524[data0524["ROUND1"] == '18대']
data_19 = data0524[data0524["ROUND1"] == '19대']

numberOfSpeaks_20 = data0524["CongOnly"].value_counts()[:30]
numberOfSpeaks_20_18 = data_18["CongOnly"].value_counts()[:30]
numberOfSpeaks_20_19 = data_19["CongOnly"].value_counts()[:30]

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20.index, y=numberOfSpeaks_20.values, title="18대, 19대 국회에서 FTA 관련 발언을 많이 한 상위 30명의 발언 횟수")
fig.show()

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20_18.index, y=numberOfSpeaks_20_18.values, title="18대 국회에서 FTA 관련 발언을 많이 한 상위 30명의 발언 횟수")
fig.show()

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20_19.index, y=numberOfSpeaks_20_19.values, title="19대 국회에서 FTA 관련 발언을 많이 한 상위 30명의 발언 횟수")
fig.show()

In [108]:
x = pd.DataFrame(sorted(dict(data_18["ROUND2"].value_counts()).items(), key=lambda x:x[0]))
x.columns = ["round2", "howmany"]

fig = px.bar(x=x["round2"], y=x["howmany"], title="18대 국회 시기별 FTA 발언 횟수")
fig.show()

In [114]:
l = []
for time in data_19.WHEN:
    if time.find("회의실") == -1:
        l.append(True)
    else:
        l.append(False)
len(l)

3372

In [115]:
data_19_erase = data_19[l]

numberOfSpeaks_20_19 = data_19["CongOnly"].value_counts()[:30]
numberOfSpeaks_20_19_erase = data_19_erase["CongOnly"].value_counts()[:30]

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20_19.index, y=numberOfSpeaks_20_19.values, title="19대 국회에서 FTA 관련 발언을 많이 한 상위 30명의 발언 횟수")
fig.show()

import plotly.express as px
fig = px.bar(x=numberOfSpeaks_20_19_erase.index, y=numberOfSpeaks_20_19_erase.values, title="시간 제거 19대 국회에서 FTA 관련 발언을 많이 한 상위 30명의 발언 횟수")
fig.show()

In [116]:
x = pd.DataFrame(sorted(dict(data_19_erase["ROUND2"].value_counts()).items(), key=lambda x:x[0]))
x.columns = ["round2", "howmany"]

fig = px.bar(x=x["round2"], y=x["howmany"], title="19대 국회 시기별 FTA 발언 횟수")
fig.show()

In [10]:
import plotly.express as px
s= data.NAME
fig = px.pie(s, values=s.value_counts()[0:30].values, names=s.value_counts()[0:30].index)
fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()

In [13]:
import plotly.express as px
s= data.COMMITTEE
fig = px.pie(s, values=s.value_counts()[0:10].values, names=s.value_counts()[0:10].index)
fig.update_traces(hoverinfo='label+percent', textinfo='value')
fig.show()

In [121]:
re.compile('[^0-9]').sub(" ", data.WHEN[0])

'2012 11 5    '

In [125]:
# import re

# def DatetimeInput(when):
#     date = []
#     for x in when:
#         date.append(re.compile('[^0-9]').sub(" ", x))
#     return date

# y = df.DateInput.value_counts().values
# x_ = df.DateInput.value_counts().index
# x = [datetime.strptime(d, "%Y %m %d ").date() for d in x_]

In [77]:
con18.rename(columns={"Unnamed: 2":"party"}, inplace=True)
con19.rename(columns={"Unnamed: 2":"party", "prep-rep":"prop_rep"}, inplace=True)

In [78]:
def makeFTA(nameList, indexList, valueList):
    cleanName = []

    for name in nameList:
        for idxCong, cong in enumerate(indexList):
            if name.find(cong) != -1:
                cleanName.append(valueList[idxCong])
                break
            elif idxCong == len(indexList) - 1:
                cleanName.append(0)
                break

    return cleanName

con18["fta"] = makeFTA(list(con18["name"]), data_18["CongOnly"].value_counts().index, data_18["CongOnly"].value_counts().values)
con19["fta"] = makeFTA(list(con19["name"]), data_19["CongOnly"].value_counts().index, data_19["CongOnly"].value_counts().values)


In [97]:
# con18.to_csv("temp18.csv")
# con19.to_csv("temp19.csv")

In [79]:
con18["where"].unique()

array(['서울특별시', '인천광역시', '경기도', '강원도', '충청북도', '대전광역시', '충청남도', '전라북도',
       '광주광역시', '전라남도', '대구광역시', '경상북도', '부산광역시', '울산광역시', '경상남도',
       '제주특별자치도', '비례대표'], dtype=object)

In [80]:
def makeWhere(whereList):
    where = []
    fr = ['서울특별시', '인천광역시', '경기도', '강원도', '충청북도', '대전광역시', '충청남도', '전라북도',
       '광주광역시', '전라남도', '대구광역시', '경상북도', '부산광역시', '울산광역시', '경상남도',
       '제주특별자치도', '세종특별자치시', '비례대표']
    to =  ['서울', '인천', '경기', '강원', '충북', '대전', '충남', '전북', '광주','전남', '대구', '경북', '부산', '울산', '경남', '제주', '세종', '비례'] 
    for place in whereList:
        if place in fr:
            where.append(to[fr.index(place)])
        else:
            where.append("None")
    return where

con18["Where"] = makeWhere(list(con18["where"]))
con19["Where"] = makeWhere(list(con19["where"]))      

In [93]:
agr = pd.read_excel("agriculture.xlsx")
man = pd.read_excel("manufacturing.xlsx")
tec = pd.read_excel("technology.xlsx")
agr2 = pd.read_excel("agriculture_level2.xlsx")
car = pd.read_excel("car_level2.xlsx")
tra = pd.read_excel("transfortation_level2.xlsx")


def makeNumOfWorkers(whereList, whereList2, numberList):
    NumOfWorkers = []

    for where in whereList:
        for idxNum, where2 in enumerate(whereList2):
            if where.find(where2) != -1:
                NumOfWorkers.append(numberList[idxNum])
                break
            elif idxNum == len(whereList2) - 1:
                NumOfWorkers.append(None)
                break

    return NumOfWorkers

con18["agr"] = makeNumOfWorkers(list(con18["Where"]), agr["행정구역별"], agr["2008"].replace("-", 0))
con18["man"] = makeNumOfWorkers(list(con18["Where"]), man["행정구역별"], man["2008"].replace("-", 0))
con18["tec"] = makeNumOfWorkers(list(con18["Where"]), tec["행정구역별"], tec["2008"].replace("-", 0))
con18["agr2"] = makeNumOfWorkers(list(con18["Where"]), agr2["행정구역별"], agr2["2008"].replace("-", 0))
con18["car"] = makeNumOfWorkers(list(con18["Where"]), car["행정구역별"], car["2008"].replace("-", 0))
con18["tra"] = makeNumOfWorkers(list(con18["Where"]), tra["행정구역별"], tra["2008"].replace("-", 0))


con19["agr"] = makeNumOfWorkers(list(con19["Where"]), agr["행정구역별"], agr["2012"].replace("-", 0))
con19["man"] = makeNumOfWorkers(list(con19["Where"]), man["행정구역별"], man["2012"].replace("-", 0))
con19["tec"] = makeNumOfWorkers(list(con19["Where"]), tec["행정구역별"], tec["2012"].replace("-", 0))
con19["agr2"] = makeNumOfWorkers(list(con19["Where"]), agr2["행정구역별"], agr2["2012"].replace("-", 0))
con19["car"] = makeNumOfWorkers(list(con19["Where"]), car["행정구역별"], car["2012"].replace("-", 0))
con19["tra"] = makeNumOfWorkers(list(con19["Where"]), tra["행정구역별"], tra["2012"].replace("-", 0))

In [94]:
pop = pd.read_excel("population.xlsx")

C:\Users\jrps2212\AppData\Local\Programs\Python\Python39\lib\site-packages\openpyxl\styles\stylesheet.py:221: UserWarning:

Workbook contains no default style, apply openpyxl's default



In [95]:
pop["Where"] = makeWhere(list(pop["행정구역(시군구)별"]))

def makeWholeWorkers(whereList, whereList2, numberList):
    WholeWorkers = []

    for where in whereList:
        for idxNum, where2 in enumerate(whereList2):
            if where.find(where2) != -1:
                WholeWorkers.append(numberList[idxNum])
                break
            elif idxNum == len(whereList2) - 1:
                WholeWorkers.append("None")
                break

    return WholeWorkers

con18["whole"] = makeWholeWorkers(list(con18["Where"]), pop["Where"], pop["2008"].replace("-", 0))
con19["whole"] = makeWholeWorkers(list(con19["Where"]), pop["Where"], pop["2012"].replace("-", 0))

In [84]:
con18["party"].value_counts()

한 새      142
통 민       77
한 무       14
무 새       14
친 새       14
자         12
한          9
민 민        8
무          6
통 무        6
무 민        6
창 무        4
민 통진       4
자 무        3
친 무        2
자 민        2
무 무        2
창조한국당      1
통진         1
무 자        1
민 통        1
친 한        1
자          1
진신 통진      1
Name: party, dtype: int64

In [85]:
con19["party"].value_counts()

새       155
민 더     101
민 국      15
통진       14
새 무      13
자 새       6
민 무       5
민 새       5
새 더       5
새         4
무 국       4
진 정       4
민 민       2
통 진       1
새 국       1
통진 무      1
무 새       1
통진 정      1
무         1
Name: party, dtype: int64

In [99]:
con19["tra"].unique()

array([2.6500e+02, 7.3400e+02, 2.0220e+03, 7.0000e+01, 4.5000e+02,
       3.9400e+02, 0.0000e+00, 7.7700e+02, 2.3490e+03, 8.1000e+01,
       1.6203e+04, 2.3010e+03, 8.8820e+03, 3.8323e+04, 8.9980e+04,
       1.1000e+01,        nan])

In [100]:
con18["agrRatio"] = con18["agr"] / con18["whole"]
con18["manRatio"] = con18["man"] / con18["whole"]
con18["tecRatio"] = con18["tec"] / con18["whole"]
con18["agr2Ratio"] = con18["agr2"] / con18["whole"]
con18["carRatio"] = con18["car"] / con18["whole"]
con18["traRatio"] = con18["tra"] / con18["whole"]

con19["agrRatio"] = con19["agr"] / con19["whole"]
con19["manRatio"] = con19["man"] / con19["whole"]
con19["tecRatio"] = con19["tec"] / con19["whole"]
con19["agr2Ratio"] = con19["agr2"] / con19["whole"]
con19["carRatio"] = con19["car"] / con19["whole"]
con19["traRatio"] = con19["tra"] / con19["whole"]

In [101]:
con18["when"] = '18대'
con19["when"] = '19대'

con = pd.concat([con18, con19])

In [102]:
def makeRuling3(partyList):
    Ruling = []

    rulingParty = ['한 새', '새 ', '새', "친 새", "무 새"]
    oppositionParty = ['통 민','민 더', '민 국']

    for party in partyList:
        if party in rulingParty:
            Ruling.append("Ruling")
        elif party in oppositionParty:
            Ruling.append("Opposition")
        else:
            Ruling.append("Others")
    
    return Ruling

con["ruling3"]=makeRuling3(con["party"])
con["ruling3"].value_counts()

Ruling        330
Opposition    193
Others        148
Name: ruling3, dtype: int64

In [103]:
def makeRuling(partyList):
    Ruling = []

    rulingParty = ['한 새', '한', '한 무', '무 새', '친 한', '자 민', '자', '무 자', '친 새', '친 무', '새 무', '새 ', '새', '민 새', '자 새', '새 국']
    oppositionParty = ['통 민', '창조한국당', '통 무', '민 민', '무', '무 민', '통진', '무 무', '진신 통진', '민 통', '민 통진', '창 무', '민 더', '민 국', '새 더', '민 무', '통 진', '무 국', '통진 정', '통진 무', '진 정']

    for party in partyList:
        if party in rulingParty:
            Ruling.append("Ruling")
        else:
            Ruling.append("Opposition")
    
    return Ruling

con["ruling"]=makeRuling(con["party"])


In [104]:
con.columns

Index(['name', 'party1', 'party', 'term_of_office', 'prop_rep', 'where',
       'loss', 'fta', 'Where', 'agr', 'man', 'tec', 'agr2', 'car', 'whole',
       'agrRatio', 'manRatio', 'tecRatio', 'agr2Ratio', 'carRatio', 'when',
       'tra', 'traRatio', 'ruling3', 'ruling'],
      dtype='object')

In [105]:
con.to_csv("con_0526.csv")

In [39]:
import plotly.express as px

fig = px.scatter(con, x='agr', y='fta', trendline="ols", title="농업 종사자 수와 FTA 발언", color="when" )
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [40]:
import plotly.express as px

fig = px.scatter(con, x='agrRatio', y='fta', trendline="ols", title="농업 종사자 비율과 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 50),
    constrain='domain'
)
fig.show()

In [41]:
import plotly.express as px

fig = px.scatter(con, x='man', y='fta', trendline="ols", title="제조업 종사자 수와 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [42]:
import plotly.express as px

fig = px.scatter(con, x='manRatio', y='fta', trendline="ols", title="제조업 종사자 비율과 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [43]:
import plotly.express as px

fig = px.scatter(con, x='tec', y='fta', trendline="ols", title="과학기술업 종사자 수와 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [44]:
import plotly.express as px

fig = px.scatter(con, x='tecRatio', y='fta', trendline="ols", title="과학기술업 종사자 비율와 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [45]:
import plotly.express as px

fig = px.scatter(con, x='agr2', y='fta', trendline="ols", title="농업(2단계 분류) 종사자 수와 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [46]:
import plotly.express as px

fig = px.scatter(con, x='agr2Ratio', y='fta', trendline="ols", title="농업(2단계 분류) 종사자 비율과 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [47]:
import plotly.express as px

fig = px.scatter(con, x='car', y='fta', trendline="ols", title="자동차 업계 종사자 수와 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [48]:
import plotly.express as px

fig = px.scatter(con, x='carRatio', y='fta', trendline="ols", title="자동차 업계 종사자 비율과 FTA 발언", color="when")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [119]:
import plotly.express as px

fig = px.box(con, x='ruling3', y='fta')
fig.update_yaxes(
    range=(-4, 25),
    constrain='domain'
)
fig.show()

In [77]:
import plotly.express as px

fig = px.box(con, x='term_of_office', y='fta')
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

In [87]:
import plotly.express as px

fig = px.box(con, x='prop_rep', y='fta')
fig.update_yaxes(
    range=(-4, 20),
    constrain='domain'
)
fig.show()

In [89]:
import plotly.express as px

fig = px.box(con, x='Where', y='fta')
fig.update_yaxes(
    range=(-4, 50),
    constrain='domain'
)
fig.show()

In [105]:
import plotly.express as px

fig = px.scatter(con, x='party', y='fta')
fig.show()

In [99]:
import plotly.express as px

fig = px.scatter(con, x='party1', y='fta')
fig.show()

In [115]:
import plotly.express as px

fig = px.box(con, x='ruling', y='fta')
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()


In [102]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='agr')
fig.show()

In [34]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='agrRatio')
fig.show()

In [103]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='man')
fig.show()

In [35]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='manRatio')
fig.show()

In [104]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='tec')
fig.show()

In [36]:
import plotly.express as px

fig = px.scatter(con, x='Where', y='tecRatio')
fig.show()

In [38]:
con_posi = con[con["fta"] != 0]

fig = px.scatter(con_posi, x='agrRatio', y='fta', trendline="ols", title="농업 종사자 비율과 FTA 발언(0 제거)")
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()

fig2 = px.scatter(con_posi, x='manRatio', y='fta', trendline="ols", title="제조업 종사자 비율과 FTA 발언(0 제거)")
fig2.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig2.show()

fig3 = px.scatter(con_posi, x='tecRatio', y='fta', trendline="ols", title="과학기술업 종사자 비율과 FTA 발언(0 제거)")
fig3.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig3.show()

fig4 = px.scatter(con_posi, x='agr2Ratio', y='fta', trendline="ols", title="농업(2단계 분류) 종사자 비율과 FTA 발언(0 제거)")
fig4.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig4.show()

fig5 = px.scatter(con_posi, x='carRatio', y='fta', trendline="ols", title="자동차 업계 종사자 비율과 FTA 발언(0 제거)")
fig5.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig5.show()

In [148]:
fig = px.box(con_posi, x='ruling', y='fta')
fig.update_yaxes(
    range=(-4, 30),
    constrain='domain'
)
fig.show()